In [37]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import seaborn as sns
from scipy.stats import norm
from numpy import cos, sin, arcsin, sqrt
from math import radians
from datetime import datetime, timedelta

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
token = 'rCsdguPf76exUsqLYxuY'

In [54]:
time_now = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%d')
url=f'http://143.198.233.67/api/v1/data_buoys?buoy=2&start_date=2021-01-01&end_date={time_now}&token={token}'
response = requests.get(url).json()
df = pd.DataFrame(response)
for i in df.columns:
    try:
        df[i] = pd.to_numeric(df[i])
    except:
        pass
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%dT%H:%M:%S.000Z')
df.sort_values('date_time', inplace=True)

In [9]:
spotter = df[['date_time', 'lat', 'lon','swvht1', 'flag_swvht1', 'swvht2', 'flag_swvht2',
       'mxwvht1', 'flag_mxwvht1', 'tp1', 'flag_tp1', 'tp2', 'flag_tp2',
       'wvdir1', 'flag_wvdir1', 'wvdir2', 'flag_wvdir2', 'wvspread1',
       'flag_wvspread1', 'pk_dir', 'flag_pk_dir', 'pk_wvspread',
       'flag_pk_wvspread', 'mean_tp', 'flag_mean_tp', 'sst',
       'flag_sst']]

In [10]:
import folium
import plotly.express as px

In [63]:
df.loc[df['flag_tp2'] > 0, 'tp2'] = np.nan


In [11]:
coordinates = []
for index, row in df.iterrows():
    coordinate = [row['lat'], row['lon']]
    coordinates.append(coordinate)

In [12]:
spotter['coordinates'] = coordinates

In [13]:
def haversine(row, lon1, lat1):
    lon2 = row['lon']
    lat2 = row['lat']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a))
    mn = 3437.9 * c
    return mn

In [14]:
spotter['distance'] = spotter.apply(lambda row: haversine(row, spotter['lon'].iloc[0], spotter['lat'].iloc[0]), axis=1)

In [15]:
spotter['veloc'] = spotter['distance'].diff()/(spotter['date_time'].diff().dt.total_seconds()/3600)

In [53]:
m = folium.Map(location=spotter['coordinates'].iloc[0], zoom_start=8)

icon = folium.features.CustomIcon(folium.Icon(color="red"),
                                      icon_size=(14, 14))

for index, row in spotter.iterrows():
    popup = str(row['date_time']) + ' - onda ' + \
     str(row['swvht1']) + 'm' + ' - veloc ' + str(round(row['veloc'],3)) + 'nós'
    folium.Marker(row['coordinates'], tooltip=popup).add_to(m)
folium.Marker(
    spotter['coordinates'].iloc[-1],
    tooltip=popup,
    icon=icon,
).add_to(m)    

folium.Circle(spotter['coordinates'].iloc[0], radius=100).add_to(m)
m

TypeError: Object of type Icon is not JSON serializable

In [17]:
spotter['distance'].tail()

578    0.649660
579    0.633098
580    0.615757
581    0.598596
582    0.581629
Name: distance, dtype: float64